In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Image-Caption-Generator'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    kaggle_api: Path
    local_files_dir: Path
    unzip_dir: Path
    batch_size: int

In [4]:
from src.ImageCaptionGenerator.constants import *
from src.ImageCaptionGenerator.utils.common import read_yaml, create_directory

True 
 True


In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.main_dir])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            kaggle_api = config.kaggle_api_key,
            local_files_dir = config.local_files_dir,
            unzip_dir = config.unzip_dir,
            batch_size = self.params.BATCH_SIZE
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.ImageCaptionGenerator import logger
from kaggle.api.kaggle_api_extended import KaggleApi
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import image
import pandas as pd
import re
import csv
from glob import glob
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import pickle
import cv2
import numpy as np
import shutil
from tqdm import tqdm

[2024-01-03 18:13:54,203: INFO: utils: NumExpr defaulting to 8 threads.]


In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        

    def download_file(self):
        try:
            dataset_name = self.config.kaggle_api
            KaggleApi().dataset_download_files(dataset_name, path=self.config.local_files_dir, force = True)
            logger.info("File downloaded successfully from Kaggle.")
        except Exception as e:
            logger.error(f"Error downloading file from Kaggle: {e}")
            raise e
        
    

    


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        filepath = os.path.join("Data","dataset.zip")
        p = os.path.join("artifact","data_ingestion","data")
        if(os.path.exists(p)):
            logger.info("files already exist")
        else:
            os.makedirs(unzip_path,exist_ok=True)
            logger.info("Extracting zip files............")
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info("Zip files extracted successfully......................")




            
        
    def image_features(self):
        # os.makedirs(self.config.local_files_dir,exist_ok=True)
        images_dir = os.path.join("artifact","data_ingestion","data","Images/")
        images = glob(images_dir+'*.jpg')
        print("Total images in dataset:",len(images))
        images_feature = {}
        base_model = ResNet50(weights='imagenet')
        model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv5_block3_out').output)

        try:
            count = 0
            with tqdm(total=len(images), desc="Converting images to features") as pbar:
                for i, image_path in enumerate(images):
                    count += 1
                    print(f"..........{count}.....{os.path.basename(image_path)}.........")
                    img = image.load_img(image_path, target_size=(224, 224))
                    img_array = image.img_to_array(img)
                    img_array = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2]))
                    img_array = preprocess_input(img_array)
                    print("Shapes: ", img_array.shape)

                    # Extract features
                    features = model.predict(img_array)

                    # Store features in the dictionary
                    images_feature[os.path.basename(image_path)] = features

                    pbar.update(1)  # Update progress bar

                print("Length of image features:", len(images_feature))
        except Exception as e:
            # Handle the exception and save the intermediate results
            print(f"Error during image processing: {e}")
            print("Saving intermediate results...")
            save_intermediate_results(images_feature)
            raise e
        finally:
            # Save the final results
            path = os.path.join("artifact", 'images_features.pkl')
            with open(path, 'wb') as f:
                pickle.dump(images_feature, f)

# ...

def save_intermediate_results(images_feature):
    intermediate_path = os.path.join("artifact", 'images_features_intermediate.pkl')
    with open(intermediate_path, 'wb') as f:
        pickle.dump(images_feature, f)



        
        

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    # data_ingestion.data_splitting()
    data_ingestion.image_features()
except Exception as e:
    raise e

[2024-01-03 18:49:40,181: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Image-Caption-Generator\config\config.yaml loaded]
[2024-01-03 18:49:40,184: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Image-Caption-Generator\params.yaml loaded]
[2024-01-03 18:49:40,186: INFO: common: Created directory at: artifact]
[2024-01-03 18:49:40,187: INFO: common: Created directory at: artifact/data_ingestion]
[2024-01-03 18:49:40,188: INFO: 2655453057: files already exist]
Total images in dataset: 8091


Converting images to features:   0%|          | 0/8091 [00:00<?, ?it/s]

..........1.....1000268201_693b08cb0e.jpg.........
Shapes:  (1, 224, 224, 3)
1/1 [==============================] - 2s 2s/step


Converting images to features:   0%|          | 1/8091 [00:01<3:59:03,  1.77s/it]

..........2.....1001773457_577c3a7d70.jpg.........
Shapes:  (1, 224, 224, 3)
1/1 [==============================] - 0s 185ms/step


...

..........8090.....997338199_7343367d7f.jpg.........
Shapes:  (1, 224, 224, 3)
1/1 [==============================] - 5s 5s/step


Converting images to features: 100%|█████████▉| 8090/8091 [5:00:52<00:04,  4.77s/it]

..........8091.....997722733_0cb5439472.jpg.........
Shapes:  (1, 224, 224, 3)
1/1 [==============================] - 5s 5s/step


Converting images to features: 100%|██████████| 8091/8091 [5:00:57<00:00,  2.23s/it]

Length of image features: 8091
